# intermediate_XED_binary
This notebook takes our custom XED binary dataset and trains an intermediate model.

## Imports & Settings

First, update working directory to parent so that we may use our custom functions

In [ ]:
import os
os.chdir('..')
# os.getcwd( )

In [ ]:
import params
from utils import *
from trainer import *

import numpy as np
import pandas as pd
from datasets import load_from_disk

from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

# suppress model warning
from transformers import logging
logging.set_verbosity_error()

# set logging level
import logging
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [ ]:
# set general seeds
set_seeds(1)

# set dataloader generator seed
g = torch.Generator()
g.manual_seed(1)

# set params for this model
params.num_labels = 2
params.output_dir = "model_saves/intermediate_XED_binary_01"

# Ensure we're on an ARM environment if necessary.
platform_check()

## Load Data

### Binary XED

In [ ]:
xed_datasets = load_from_disk("data/inter_XED/itesd_xed_binary_balanced.hf")
xed_datasets

In [ ]:
# we will need to view and prep the datasets
# this is more easily done as dataframes
xed_train_df = xed_datasets['train'].to_pandas()
xed_validate_df = xed_datasets['validation'].to_pandas() 

In [ ]:
# view training dataset
print("train_df Info:")
print(xed_train_df.info())
print("\ntrain_df Value Counts")
print(xed_validate_df['label'].value_counts())

In [ ]:
# view validation dataset
print("validate_df Info:")
print(xed_validate_df.info())
print("\n validate_df Value Counts")
print(xed_validate_df['label'].value_counts())

## Preprocess

In [ ]:
train_token_id = []
train_attention_masks = []

# encode training text
for sample in xed_train_df.text.values:
  encoding_dict = preprocessing_dyna(sample, params.tokenizer)
  # parse encoding dict to lists
  train_token_id.append(encoding_dict['input_ids']) 
  train_attention_masks.append(encoding_dict['attention_mask'])

In [ ]:
validate_token_id = []
validate_attention_masks = []

# encode validation text 
for sample in xed_validate_df.text.values:
  encoding_dict = preprocessing_dyna(sample, params.tokenizer)  
  # parse encoding dict to lists
  validate_token_id.append(encoding_dict['input_ids']) 
  validate_attention_masks.append(encoding_dict['attention_mask'])

In [ ]:
train_features = []
for i in range(len(train_token_id)):
    train_features.append({'label': xed_train_df.label.values[i], 
                           'input_ids': train_token_id[i], 
                           'attention_mask':train_attention_masks[i]})

validate_features = []
for i in range(len(validate_token_id)):
    validate_features.append({'label': xed_validate_df.label.values[i], 
                              'input_ids': validate_token_id[i],
                              'attention_mask':validate_attention_masks[i]})

## Data Split
We split the dataset into train (80%) and validation (20%) sets, and wrap them around a torch.utils.data.DataLoader object.

In [ ]:
# Prepare DataLoader
train_dataloader = DataLoader(
            train_features,
            sampler = RandomSampler(train_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=seq_class_collate
        )

validation_dataloader = DataLoader(
            validate_features,
            sampler = RandomSampler(validate_features),
            batch_size = params.batch_size,
            worker_init_fn=seed_worker,
            generator=g,
            collate_fn=seq_class_collate
        )

In [ ]:
# view an example from the dataloader
next(iter(train_dataloader))

## Train

Download transformers.RobertaForSequenceClassificatio, which is a RoBERTa model with a linear layer for sentence classification (or regression) on top of the pooled output:

In [ ]:
# Load the RobertaForSequenceClassification model
model = RobertaForSequenceClassification.from_pretrained('roberta-base',
                                                         num_labels = params.num_labels,
                                                         output_attentions = False,
                                                         output_hidden_states = False,
                                                         )

from torchinfo import summary
summary(model, input_size=(1, 512), dtypes=['torch.IntTensor'])

Set model to device, initialize trainer

In [ ]:
model.to(params.device)
print(f"Device: {params.device}")

optimizer = torch.optim.Adam(params=model.parameters(), lr=params.learning_rate) #roberta

trainer = Trainer(model=model,
                  device=params.device,
                  tokenizer=params.tokenizer,
                  train_dataloader=train_dataloader,
                  validation_dataloader=validation_dataloader,
                  epochs=params.epochs,
                  optimizer=optimizer,
                  val_loss_fn=params.val_loss_fn,
                  num_labels=params.num_labels,
                  output_dir=params.output_dir,
                  save_freq=params.save_freq,
                  checkpoint_freq=params.checkpoint_freq)

Fit the model to our training data.

In [ ]:
print(params.max_length)
trainer.fit()